In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.7`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.apache.spark.sql._
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}
import org.apache.spark.sql.functions._

val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Dataframe API")
                .config("spark.log.level", "WARN")
                .getOrCreate()

import spark.implicits._

println(s"spark.version == ${spark.version}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/09 16:10:29 INFO SparkContext: Running Spark version 3.5.7
26/02/09 16:10:29 INFO SparkContext: OS info Linux, 6.8.0-90-generic, amd64
26/02/09 16:10:29 INFO SparkContext: Java version 1.8.0_442
26/02/09 16:10:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".


spark.version == 3.5.7


import $ivy.$
import org.apache.log4j.{Level, Logger}
import org.apache.spark.sql._
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}
import org.apache.spark.sql.functions._
spark: SparkSession = org.apache.spark.sql.SparkSession@16882580
import spark.implicits._

In [27]:
case class TaxiTrip(PULocationID: Int, trip_distance: Double)
case class TaxiZone(LocationID: Int, Borough: String, Zone: String)

defined class TaxiTrip
defined class TaxiZone

In [29]:
val tripsDS = spark.read.parquet("data/yellow_taxi_jan_25_2018/")
.select($"PULocationID",$"trip_distance").as[TaxiTrip]

val zonesDS = spark.read
  .option("header", "true")
  // .option("inferSchema", "true")
  .csv("data/taxi_zones.csv").select($"LocationID", $"Borough", $"Zone")
  .withColumn("LocationID", col("LocationID").cast(IntegerType))
  .as[TaxiZone]


tripsDS: Dataset[TaxiTrip] = [PULocationID: int, trip_distance: double]
zonesDS: Dataset[TaxiZone] = [LocationID: int, Borough: string ... 1 more field]

In [42]:
tripsDS.where($"trip_distance" === 0.0).count()

res42: Long = 1870L

In [30]:
tripsDS.printSchema()
tripsDS.count()
tripsDS.show(2)
zonesDS.printSchema()
zonesDS.count()
zonesDS.show(2)

root
 |-- PULocationID: integer (nullable = true)
 |-- trip_distance: double (nullable = true)

+------------+-------------+
|PULocationID|trip_distance|
+------------+-------------+
|          48|         2.02|
|          79|        10.13|
+------------+-------------+
only showing top 2 rows

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)

+----------+-------+--------------+
|LocationID|Borough|          Zone|
+----------+-------+--------------+
|         1|    EWR|Newark Airport|
|         2| Queens|   Jamaica Bay|
+----------+-------+--------------+
only showing top 2 rows



res30_1: Long = 331893L
res30_4: Long = 265L

In [39]:
val resultDS =
  zonesDS.join(tripsDS, zonesDS("LocationID") === tripsDS("PULocationID"), "left")
    .select($"Borough", $"Zone", $"trip_distance")

  .groupBy(
      col("Borough"),
    col("Zone")
    
  )
    .agg(
      count("trip_distance").as("trip_count"),
      min("trip_distance").as("min_distance"),
      avg("trip_distance").as("avg_distance"),
      max("trip_distance").as("max_distance"),
      stddev("trip_distance").as("stddev_distance")
    )

resultDS: DataFrame = [Borough: string, Zone: string ... 5 more fields]

In [40]:
resultDS.printSchema()
resultDS.count()
resultDS.show(5)

root
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- trip_count: long (nullable = false)
 |-- min_distance: double (nullable = true)
 |-- avg_distance: double (nullable = true)
 |-- max_distance: double (nullable = true)
 |-- stddev_distance: double (nullable = true)

+---------+--------------------+----------+------------+------------------+------------+------------------+
|  Borough|                Zone|trip_count|min_distance|      avg_distance|max_distance|   stddev_distance|
+---------+--------------------+----------+------------+------------------+------------+------------------+
|   Queens|            Glendale|         3|         0.8|1.8966666666666665|        3.59|1.4876267453004917|
| Brooklyn|    Bensonhurst West|         5|         0.0|             1.238|         2.6|1.0743463128805348|
|   Queens|        North Corona|         8|         0.0|            3.0175|        6.98|2.7247109938487055|
| Brooklyn|     Windsor Terrace|        10|        

res40_1: Long = 262L

In [35]:
resultDS.where($"trip_count" === 0).show()

+-------------+--------------------+----------+------------+------------+------------+---------------+
|      Borough|                Zone|trip_count|min_distance|avg_distance|max_distance|stddev_distance|
+-------------+--------------------+----------+------------+------------+------------+---------------+
|       Queens|           Woodhaven|         0|        NULL|        NULL|        NULL|           NULL|
|Staten Island|       Arden Heights|         0|        NULL|        NULL|        NULL|           NULL|
|        Bronx|         Eastchester|         0|        NULL|        NULL|        NULL|           NULL|
|Staten Island|Saint George/New ...|         0|        NULL|        NULL|        NULL|           NULL|
|        Bronx|        Country Club|         0|        NULL|        NULL|        NULL|           NULL|
|       Queens|       College Point|         0|        NULL|        NULL|        NULL|           NULL|
|       Queens|           Glen Oaks|         0|        NULL|        NULL|

In [43]:
resultDS.write.mode("overwrite").parquet("/tmp/output/taxi_zone_stats")